# This code compute the signal-to-noise of the matter power spectrum with and without non-Gaussian covariance and compares to simulations

#### import packages

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import SymLogNorm
from joblib import Parallel, delayed
from scipy.interpolate import InterpolatedUnivariateSpline as interpolate
from cubature import cubature

%config InlineBackend.figure_format = 'svg' 
plt.rcParams['mathtext.fontset'] = 'cm'
plt.rcParams.update({'font.size': 14})

## Choose the redshift to analyse and load the power spectra

In [ ]:
zs = [0,3,10,30,50,100]

zi = 5 #CHOOSE THE INDEX OF THE ABOVE REDSHIFTS HERE
z = zs[zi]
print("z =",z)
PLin = np.loadtxt("PLinear.dat")
kLin = PLin[:,0]
PLin = PLin[:,zi+1]
PLinInt = interpolate(kLin,PLin,ext=1)

if z == 10 :
    PNonLin = np.load('Pks_z10.npy')
    kNonLin = PNonLin[0,:]
    PNonLin = PNonLin[1:].mean(0)
    PNonLinInt = interpolate(kNonLin,PNonLin,ext=0)
    
    kF = 2.*np.pi/250.
    nbar = 512.**3 / 250.**3
    
elif z > 10:
    PNonLin = np.loadtxt('PHalo.dat')
    kNonLin = PNonLin[:,0]
    PNonLin = PNonLin[:,zi+1]
    PNonLinInt = interpolate(kNonLin,PNonLin,ext=2)
    
    kF = 2.*np.pi/1000.
    nbar = 512**3 / 1000**3
    
else:
    PNonLin = np.loadtxt('PNonLinearQuijote.dat')
    kNonLin = PNonLin[:,0]
    if z == 0:
        PNonLin = PNonLin[:,1]
    if z == 3:
        PNonLin = PNonLin[:,-1]
    PNonLinInt = interpolate(kNonLin,PNonLin,ext=0)

    kF = 2.*np.pi/1000.
    nbar = 512**3 / 1000**3
    
kNL = (np.trapz(PLin,kLin)/6/np.pi**2)**-.5
kNL 
    
if z < 3:
    kmax = 0.1
elif z == 3:
    kmax = 0.11
else:
    kmax = 0.5
    
plt.loglog(kLin,PLin,label='linear power spectrum')
plt.loglog(kNonLin,PNonLin,label='measured power spectrum')
plt.loglog(kLin,np.ones_like(kLin)/nbar,label='shot noise')
plt.xlabel("$k$ $[h/Mpc]$")
plt.ylabel("$P(k)$ $[(Mpc/h)^3]$")
plt.grid()
plt.legend(fontsize=13)
plt.show()

kGrid = kNonLin[kNonLin < kmax]
PLinGrid = PLinInt(kGrid)
PNonLinGrid = PNonLinInt(kGrid)
Vi = 4*np.pi*kGrid**2 * kF

### Get the full covariance from simulations

In [ ]:
N = 14999
if z > 3:
    QuijoteData = np.load('Pks_z10.npy').T
    kQuij = QuijoteData[:,0]

else:
    QuijoteData = np.load(f"Quijote-z={z}.npy")
    kQuij = QuijoteData[:,0]

kGrid = kQuij[kQuij<kmax]
Pks = QuijoteData[kQuij < kmax,1:N]
PLinGrid = PLinInt(kGrid)
print(Pks.shape)

Cov_Quij = np.cov(Pks)

Cov_Quij_Inv = np.linalg.inv(Cov_Quij)

N_sims = len(Pks.T)
hartlapfactor = (N_sims - len(Cov_Quij) - 2) / (N_sims - 1)
Cov_Quij_Inv *= hartlapfactor

r_Quij = np.zeros_like(Cov_Quij)

r_Quij = Cov_Quij / np.outer(np.diag(Cov_Quij),np.diag(Cov_Quij))**.5

plt.imshow(r_Quij,origin='lower',cmap='RdBu',vmin=-1,vmax=1)
plt.colorbar()
plt.show()

plt.imshow(r_Quij,cmap='RdBu',origin='lower',norm=SymLogNorm(linthresh=0.01, linscale=0.01,vmin=-1,vmax=1))
plt.colorbar()
plt.show()

### Compute the signal-to-noise with Gaussian and full covariance

In [ ]:
def SNL_sims(i):
    Pbins = PNonLinGrid[:i]
    
    C_Quij = Cov_Quij[:i,:i]
    C_Quij_Inv = np.linalg.inv(C_Quij)
    N_sims = len(Pks.T)
    hartlapfactor = (N_sims - len(C_Quij) - 2) / (N_sims - 1)
    C_Quij_Inv *= hartlapfactor
    
    C_G = np.diag(np.diag(C_Quij))
    C_G_Inv = np.linalg.inv(C_G)
    
    SN2NG = np.dot(np.dot(Pbins,C_Quij_Inv),Pbins)
    SN2G = np.dot(np.dot(Pbins,C_G_Inv),Pbins)
    
    return SN2G, SN2NG

kSims = kGrid+kF/2
SN2Sims = np.array(Parallel(n_jobs=-1,verbose=3)([delayed(SNL_sims)(i) for i in range(1,len(kGrid)+1)]))

## Now we perform the calculation using weighted estimators. First we compute the signal (which is equivalent to the noise when only considering Gaussian covariance):

$$ \langle \hat{S} \rangle = \langle N^2 \rangle_G = \frac{(2\pi)^3}{k_F^3}\int_{\textbf{k}_{1}} $$

In [ ]:
# define the integrand
def SignalIntegrand(k):
    k1 = k[:,0]
    
    dVol = 4*np.pi * k1**2   # 4pi^2 comes from the angular integration of d^3k_1, k_1^2 is just the jacobian for spherical coordinates

    P1 = PLinInt(k1)
    
    P1NL = PNonLinInt(k1)

    W = P1NL / 2 / P1NL / P1NL
    
    result = W * P1NL
    result *= dVol
    return result / kF**3

#perform the integral
def Signal(kmin,kmax,NL=False):
    
    def BatchIntegrand(k):
        return SignalIntegrand(k)

    val, err = cubature(BatchIntegrand, 1, 1, [kmin], [kmax],vectorized=True,abserr=1e-100,relerr=1e-10)

    return val[0]

%time Signal(kGrid[0],kGrid[-1])

### the non-Gaussian noise contribution from the trispectrum reads: 

$$ \langle N^2 \rangle_T = \frac{(2\pi)^3}{k_F^3} \int_{\textbf{k}_{1},\textbf{q}_{1}} \frac{T_\delta(\textbf{k}_{1},-\textbf{k}_{1},\textbf{q}_{1},-\textbf{q}_{1})}{4P_\delta(k_1)P_\delta(q_1)}$$

In [ ]:
def NoiseTIntegrand(k,kmin,kmax):
    k1 = kmin + k[:,0]*(kmax-kmin)
    k2 = kmin + k[:,1]*(kmax-kmin)

    theta = k[:,2]
    
    k1k2 = k1*k2*np.cos(theta)
    
    K12 = np.sqrt(k1**2 + k2**2 + 2*k1k2)
    K1m2 = np.sqrt(k1**2 + k2**2 - 2*k1k2)

    #4pi comes from aligning k1 with z axis and doing both angular integrals
    #2pi comes from doing the phi2 integral, so only the theta angle remains
    dVol = 4*np.pi * 2*np.pi * k1**2 * k2**2 * np.sin(theta) * (kmax-kmin) * (k1-kmin)

    Pk1 = PLinInt(k1)
    Pk2 = PLinInt(k2)
    PK1p2 = PLinInt(K12)
    PK1m2 = PLinInt(K1m2)
    
    Pk1NL = PNonLinInt(k1)
    Pk2NL = PNonLinInt(k2)
    
    W1 = Pk1NL / 2 / Pk1NL / Pk1NL
    W2 = Pk2NL / 2 / Pk2NL / Pk2NL

    F3122 = (-2*(k1**3 - k1*K12**2)**2*(11*k1**2*K12**2 - 4*K12**4 + 3*k1**2*K1m2**2 + 53*K12**2*K1m2**2) + 
  (k1 - K12)*(k1 + K12)*(97*k1**4*K12**2 - 107*k1**2*K12**4 + 28*K12**6 + (-3*k1**4 - 337*k1**2*K12**2 + 70*K12**4)*K1m2**2)*k2**2 + 
  (565*k1**4*K12**2 - 546*k1**2*K12**4 + 133*K12**6 + (45*k1**4 - 244*k1**2*K12**2 + 161*K12**4)*K1m2**2)*k2**4 + 
  (k1**2*(403*K12**2 - 57*K1m2**2) - 14*(13*K12**4 + 8*K12**2*K1m2**2))*k2**6 + 7*(11*K12**2 + 3*K1m2**2)*k2**8)/(3024*k1**2*K12**2*K1m2**2*k2**4)

    F3211 = (7*(k1**3 - k1*K12**2)**2*(11*k1**2*K12**2 - 4*K12**4 + 3*k1**2*K1m2**2 - 10*K12**2*K1m2**2) + 
  (k1 - K12)*(k1 + K12)*(403*k1**4*K12**2 - 143*k1**2*K12**4 - 8*K12**6 + (-57*k1**4 - 301*k1**2*K12**2 + 106*K12**4)*K1m2**2)*k2**2 + 
  (565*k1**4*K12**2 - 204*k1**2*K12**4 - 38*K12**6 + (45*k1**4 - 334*k1**2*K12**2 + 206*K12**4)*K1m2**2)*k2**4 + (52*K12**4 - 94*K12**2*K1m2**2 + k1**2*(97*K12**2 - 3*K1m2**2))*k2**6 - 
  2*(11*K12**2 + 3*K1m2**2)*k2**8)/(3024*k1**4*K12**2*K1m2**2*k2**2)

    F211m2 = (-5*(k1**2 - K1m2**2)**2 + 3*(k1**2 + K1m2**2)*k2**2 + 2*k2**4)/(28*k1**2*K1m2**2)
    F221m2 = (2*k1**4 - 5*(K1m2**2 - k2**2)**2 + 3*k1**2*(K1m2**2 + k2**2))/(28*K1m2**2*k2**2)
    F211p2 = (-5*(k1**2 - K12**2)**2 + 3*(k1**2 + K12**2)*k2**2 + 2*k2**4)/(28*k1**2*K12**2)
    F221p2 = (2*k1**4 - 5*(K12**2 - k2**2)**2 + 3*k1**2*(K12**2 + k2**2))/(28*K12**2*k2**2)

    T = 12*F3211*Pk1*Pk1*Pk2 
    T += 12*F3122*Pk1*Pk2*Pk2
    T += 4 * (F211m2*Pk1 + F221m2*Pk2)**2 * PK1m2 
    T += 4 * (F211p2*Pk1 + F221p2*Pk2)**2 * PK1p2
    
    result = W1 * W2 * T / (2*np.pi)**3 / kF**3
    result *= dVol
    return result

def NoiseT(kmin,kmax,rtol=1e-4):
    
    def integrandbatch(k):
        return NoiseTIntegrand(k,kmin,kmax)

    val, err = cubature(integrandbatch, 3, 1, [0,0,2e-8], [1,1,np.pi-2e-8],vectorized=True,abserr=1e-100,relerr=rtol)
    
    return val[0]

%time NoiseT(kGrid[0],kGrid[-1])

In [ ]:
kmaxs_full = np.logspace(np.log10(kF*2),np.log10(kNL))
# kmaxs_full = np.logspace(np.log10(kF*1.1),0)

S_full = np.array([Signal(kF,kmaxy,NL=False) for kmaxy in kmaxs_full])
SNL_full = np.array([Signal(kF,kmaxy,NL=True) for kmaxy in kmaxs_full])
NT_full = np.array(Parallel(n_jobs=-1,verbose=3)([delayed(NoiseT)(kF,kmaxy) for kmaxy in kmaxs_full]))

SN2G_Full = S_full
SN2NG_Full= SNL_full**2 / (SNL_full+NT_full)

In [ ]:
R_Sims = (SN2Sims[:,1]/SN2Sims[:,0])**.5
R_Full = (SN2NG_Full/SN2G_Full)**.5

In [ ]:
plt.loglog(kSims,SN2Sims[:,1]**.5,".")
plt.loglog(kSims,SN2Sims[:,0]**.5,".")
plt.gca().set_prop_cycle(None)
plt.loglog(kmaxs_full,SN2NG_Full**.5)
plt.loglog(kmaxs_full,SN2G_Full**.5)
plt.xlabel("$k_{max}$ $[h/Mpc]$")
plt.ylabel("S/N")
plt.grid()
plt.show()

plt.semilogx(kmaxs_full,R_Full)
plt.gca().set_prop_cycle(None)
plt.semilogx(kSims,R_Sims,".")
plt.vlines(kNL,0,1,linestyle="--")
plt.ylim(0.7,1)
plt.xlim(1e-2,kmaxs_full[-1])
plt.grid(linestyle=":")
plt.ylabel("$(S/N)_{NG} / (S/N)_{G}$")
plt.xlabel("$k_{max}$ [h/Mpc]")
plt.show()

In [ ]:
if z < 30: np.save(f"Results/PS_R_Sims_z{z}",np.array([kSims,R_Sims]).T)
np.save(f"Results/PS_R_Full_z{z}",np.array([kmaxs_full,R_Full]).T)

In [ ]:
zs = [0,3,10,30,50,100]

fig, axs = plt.subplots(1,1,sharex=True,figsize=(8,5))
plt.subplots_adjust(wspace=0, hspace=0.125)

for zi,zz in enumerate(zs):
    R_Full = np.load(f"Results/PS_R_Full_z{zz}.npy")
    k_Full = R_Full[:,0]
    R_Full = R_Full[:,1]
    plt.semilogx(k_Full,R_Full,linewidth=2)

plt.gca().set_prop_cycle(None)

for zi,zz in enumerate(zs):
    if zz > 10: continue
    R_Sims =np.load(f"Results/PS_R_Sims_z{zz}.npy")
    k_Sims = R_Sims[:,0]
    R_Sims = R_Sims[:,1]
    plt.semilogx(k_Sims,R_Sims,"d",markeredgecolor='black')
    
plt.ylim(0.75,1)
plt.xlim(1e-2,2e1)
# plt.yticks([0.1*n for n in range(11)])
plt.grid(linestyle=":")
plt.xlabel("$k_{max}$ $[h/Mpc]$",fontsize=17)
plt.ylabel("$(S/N)_{NG} / (S/N)_{G}$",fontsize=17)
plt.legend(np.append([f"$z = {zz}$" for zz in zs],"simulation"),loc=3,fontsize=14)
plt.title("Matter Power Spectrum",fontsize=17)
plt.savefig("Results/R_PS.pdf",format='pdf',bbox_inches='tight')
plt.show()